In [1]:
from readers.ria_reader import ria_reader
from datasets.gen_title_dataset import GenTitleDataset
from models.bottleneck_encoder_decoder import BottleneckEncoderDecoderModel
from transformers import AutoTokenizer, EncoderDecoderModel, EncoderDecoderConfig, \
    BertConfig, PreTrainedModel, PretrainedConfig, \
    Trainer, TrainingArguments, logging

In [2]:
logging.set_verbosity_info()

In [28]:
import json
import random
import copy
import torch

In [4]:
from _jsonnet import evaluate_file as jsonnet_evaluate_file

In [13]:
config = json.loads(jsonnet_evaluate_file('../configs/gen_title.jsonnet'))

In [6]:
model_path = '/Users/leshanbog/Documents/neural_models/rubert_cased_L-12_H-768_A-12_pt/'

In [10]:
model = BottleneckEncoderDecoderModel.from_encoder_decoder_pretrained(model_path, model_path)

loading configuration file /Users/leshanbog/Documents/neural_models/rubert_cased_L-12_H-768_A-12_pt/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "type_vocab_size": 2,
  "vocab_size": 119547
}

loading weights file /Users/leshanbog/Documents/neural_models/rubert_cased_L-12_H-768_A-12_pt/pytorch_model.bin
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel 

Set `config.is_decoder=True` and `config.add_cross_attention=True` for decoder_config


In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_path, do_lower_case=False)

loading configuration file /Users/leshanbog/Documents/neural_models/rubert_cased_L-12_H-768_A-12_pt/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "type_vocab_size": 2,
  "vocab_size": 119547
}

Model name '/Users/leshanbog/Documents/neural_models/rubert_cased_L-12_H-768_A-12_pt/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-

In [16]:
model.eval();

In [55]:
txt = 'фыыф ыфв'

In [56]:
inp = torch.LongTensor(tokenizer.encode(txt)).unsqueeze(0)
otpt = model.encoder(inp)['last_hidden_state']

In [57]:
otpt[0][0].shape

torch.Size([768])